## Creating a User Database with CSV Files

### Final Design

This week’s project is all about logic. We need to understand how to set up a step-by-step
process for logging users in and out.

There are three main parts to this program, registering
a user, logging a user in, and the main loop that will run the program.

Knowing that the irst two are tasks, we can make functions out of them and call them when necessary in the
main loop. Let’s go ahead and lay out the logical process for this program:

```
1. Check to see if user is logged in.
    a. If logged in, ask if they would like to log out/quit.
        i. Either quit or log out user and restart.
    b. Else, ask if they would like to log in/register/quit.
        i. If log in, ask user for e-mail/password.
            1. If correct, log user in and restart.
            2. Else, display error and restart.
        ii. If register, ask for e-mail/password/password2.
            1. If passwords match, save user and restart.
            2. Else, display error and restart.
        iii. If quit, say thank you and exit program.
```

In [17]:
# import necessary packages
import csv
import re
from IPython.display import clear_output
from getpass import getpass

# ------------- helper functions ------------- # 4
def is_valid_email(email: str) -> bool: # 2
    """
    Returns a bool indicating a match against a universal email pattern.
    """
    PATTERN = re.compile(r"\b[\w\.]+@[\w]+\.[a-zA-Z]{2,}\b")
    return bool(re.match(PATTERN, email))

def is_registered_email(email: str) -> bool:
    """
    Checks whether an email is already registered in the database
    """
    with open("users.csv", mode = "r") as f:
        reader = csv.reader(f, delimiter = ",")
        return any(email == row[0] for row in reader)

def matching_passwords(pass1: str, pass2: str) -> bool:
    return pass1 == pass1

def correct_password(email: str, password: str) -> bool:
    """
    Checks if the given password is associated with the registered email
    """
    with open("users.csv", mode = "r") as f:
        reader = csv.reader(f, delimiter = ",")
        for row in reader:
            if row[0] == email:
                if row[1] == password:
                    return True        
        return False
    
def logout_user() -> bool:
    """
    Logs the user out if logged in.
    """
    global logged_in
    if logged_in:
        print("Your are now logged out!")
        return not logged_in
    else:
        print("You are already not logged in!")
        return logged_in

# ------------- handling user registration -------------
def register_user() -> bool:
    with open("users.csv", mode = "a", newline = "") as f:
        # loop to ensure valid credentials are met.
        flag = False
        while not flag:
            user_confirm = input("Press any key to begin registering you info, 'quit' to quit: ").casefold()
            
            if user_confirm == "quit": 
                break
            
            # verify the email
            email = input("E-mail: ")
            if not is_valid_email(email):
                print("Invalid email submitted. Try again or quit.")
                continue
            if is_registered_email(email):
                print(f"{email} is already in our database. Enter a different email")
                continue
            
            # verify the password
            password1 = getpass("Password: ") # 5
            password2 = getpass("Re-type password: ") # 5
            if not matching_passwords(password1, password2):
                print("Non-matching passwords enterred. Try again or quit!")
                continue
                
            # if the flow of control has reached this far, we have obtained value
            # credentials from the user - prep the csv.writer and update database
            writer = csv.writer(f, delimiter = ",")
            writer.writerow([email, password1])
            print("Successful registry!")
            flag = True
        return False # this is for the 'logged_in' variable in the main loop
            
# ------------- handling user login -------------
def login_user() -> bool:
    flag = False
    while not flag:
        user_confirm = input("Press any key to begin logging in, 'quit' to quit: ").casefold()
        
        # terminal condition
        if user_confirm == "quit":
            break
            
        # verify the email
        user_email = input("E-mail: ")
        if not is_valid_email(user_email):
            print("Invalid email submitted. Try again or quit.")
            continue
        # this time having a registered email is a good thing.
        if not is_registered_email(user_email):
            print(f"{user_email} is not recognized in our database. Please enter a registered email or sign up.")
            continue
            
        # if the flow of control has reached this stage, we have a valid and registered 
        # email on our hands. Query the user for the corresponding password and verify its validity.
        user_password = getpass(f"Enter the password for {user_email}: ") # 5
        if correct_password(user_email, user_password):
            print("You are now logged in!")
            flag = True
            return True
        else:
            print("Incorrect password! Try again from the top!")
            continue
            
# ------------- creating the main loop -------------
# variables for main loop
active = True
logged_in = False
CHOICES = ['logout', 'login', 'register', 'quit']
mapping = {'login': login_user,
           'logout': logout_user,
           'register': register_user} # 7

# main loop
while active:
    if logged_in:
        print("1. Logout")
        print("2. Quit")
    else:
        print("1. Login")
        print("2. Register")
        print("3. Quit")
        
    user_choice = input("What would you like to do? ").lower()
    
    # terminal clause   # 3
    if user_choice == "quit":
        active = False
        print("Thank you for using our program!")
        break
    
    # clause for invalid inputs
    if user_choice not in CHOICES:
        print("Please select a valid choice.")
        continue # 6
        
    # -- primary business logic --
    logged_in = mapping[user_choice]() # 7

1. Login
2. Register
3. Quit


What would you like to do?  Login
Press any key to begin logging in, 'quit' to quit:  f
E-mail:  noorudin_buraleh@hotmail.co.uk
Enter the password for noorudin_buraleh@hotmail.co.uk:  ·········


You are now logged in!
1. Logout
2. Quit


What would you like to do?  logout


1. Login
2. Register
3. Quit


What would you like to do?  quit


Thank you for using our program!


### Summary of Changes I've Made from Milliken:

1. Used a more concise form of membership testing to verify that the user is logging in with a registered and valid email/password
2. Used regular expression pattern matching with the re module to verify the validity of an input email.
3. Moved the terminal and logout conditions above the primary business logic section.
4. Encapsulated the logic for handling valid credentials to a series of separate helper functions, in line with the single responsibility principle.
5. Used the getpass function from the built-in module getpass, to give the user a more familiar password-entering experience.
6. Verified that the user made a valid, available choice in the main loop.
7. Condensed the primary business logic to a single line by encapsulating all functionality to functions associated with user input by a dictionary